# Garen Optimization
Garen is a champion from popular video game League of Legends. </br>
He is known for his high damage output and survivability alongside a unique playstyle. In this project, we will attempt to optimize his build using machine learning. </br>
Data taken at patch 25.24
## Abilities introduction
### Q: Decisive Strike
gives garen movespeed and an empowered basic attack, silencing the enemy.
### W: Courage
defensive ability which gives Garen a shield, tenacity and a percentage of incoming damage reduction.
### E: Judgement
core ability which allows garen to spin and deal continuous AOE damage and extra to the closest target.
### R: Demacian Justice
deals execute damage to one single enemy.
</br>
Now let's define the ability functions which we can input the stats to get the damage of the ability (excluding Q and R for later)

In [4]:
import pandas as pd
import numpy as np

def q_damage(
    q_level: int,
    stats: dict[str, float],
    target_armor: float = 0,
    target_magic_resistance: float = 0,
    crit_multiplier: float = 1.75,

):
    q_level_damage = [30, 60, 90, 120, 150]
    q_damage = q_level_damage[q_level - 1] + stats["ad"] * 0.5
    crit_adjusted_q_damage = q_damage * (crit_multiplier * stats["crit_chance"] + (1-stats["crit_chance"]))
    return crit_adjusted_q_damage

def e_turn_damage(
    e_level: int,
    stats: dict[str, float],
    target_armor: float = 0,
    target_magic_resistance: float = 0,
    crit_multiplier: float = 1.4, # if IE built then 1.72
    # reminder: if hit by e spins 6 times, armour is reduced by 25%
    # garen spins around 7 + 1 per 25% bonus attack speed times over 3 seconds
):
    e_level_damage_far = [4, 8, 12, 16, 20]
    e_level_ad_scaling_far = [0.36, 0.38, 0.4, 0.42, 0.44]
    e_level_damage_near = [5, 10, 15, 20, 25]
    e_level_ad_scaling_near = [0.45, 0.475, 0.5, 0.525, 0.55]
    e_near_dmg = e_level_damage_near[e_level - 1] + stats["ad"] * e_level_ad_scaling_near[e_level - 1]
    e_far_dmg = e_level_damage_far[e_level - 1] + stats["ad"] * e_level_ad_scaling_far[e_level - 1]
    
    crit_adjust = crit_multiplier * stats["crit_chance"] + (1-stats["crit_chance"])

    [near_dmg, far_dmg] = [e_near_dmg, e_far_dmg]*crit_adjust
    return [near_dmg, far_dmg]

def r_damage(
    r_level: int,
    percentage_dmg_multiplier: float,
    target_health: float,
    target_max_health: float,
):
    r_level_damage = [150, 250, 350]
    r_level_punish = [0.25, 0.3, 0.35]
    missing_health = target_max_health - target_health
    r_damage = r_level_damage[r_level - 1] + percentage_dmg_multiplier * missing_health * r_level_punish[r_level - 1]
    return r_damage

In [5]:
def calculate_stat(base, growth, level):
    if level == 1:
        return base
    # Official League of Legends growth formula
    multiplier = (level - 1) * (0.7025 + 0.0175 * (level - 1))
    return round(base + growth * multiplier, 2)

levels = np.arange(1, 19)

garen_stats = pd.DataFrame({
    "level": levels,
    "ad": [calculate_stat(69, 4.5, lvl) for lvl in levels],
    "crit_chance": [0] * 18,
    "health": [calculate_stat(690, 98, lvl) for lvl in levels],
    "armor": [calculate_stat(38, 4.2, lvl) for lvl in levels],
    "magic_resistance": [calculate_stat(32, 1.55, lvl) for lvl in levels],
    "base_attack_speed": [0.625] * 18,
    "bonus_attack_speed": [round((lvl - 1) * (0.7025 + 0.0175 * (lvl - 1)) * 0.0365, 4) for lvl in levels]
})

garen_stats.head()

,level,ad,crit_chance,health,armor,magic_resistance,base_attack_speed,bonus_attack_speed
0,1,69.00,0,690.00,38.00,32.00,0.625,0.0000
1,2,72.24,0,760.56,41.02,33.12,0.625,0.0263
2,3,75.64,0,834.55,44.20,34.29,0.625,0.0538
3,4,79.19,0,911.97,47.51,35.51,0.625,0.0827
4,5,82.90,0,992.82,50.98,36.79,0.625,0.1128


In [ ]:
# testing block
q_damage(q_level = 1, stats = garen_stats)

0      64.500
1      66.120
2      67.820
3      69.595
4      71.450
5      73.390
6      75.400
7      77.495
8      79.665
9      81.915
10     84.245
11     86.650
12     89.140
13     91.700
14     94.345
15     97.070
16     99.870
17    102.750
dtype: float64

In [ ]:
from items import shop_item
beserkers_greaves = shop_item(
    name="Beserker's Greaves",
    price = 1100,
    stats = {
        "bonus_attack_speed": 45,
        "bonus_armor": 25
    })

dorans_blade = shop_item(
    name="Doran's Blade",
    price = 450,
    stats = {
        "ad": 10,
        "health": 80,
    })

dorans_shield = shop_item(
    name="Doran's Shield",
    price = 450,
    stats = {
        "health": 110,
    })

phantom_dancer = shop_item(
    name="Phantom Dancer",
    price = 2650,
    stats = {
        "crit_chance": 0.25,
        "bonus_attack_speed": 0.6,
        "percent_move_speed": 0.08
    })

stride_breaker = active_item(
    name="Stride Breaker",
    price = 3300,
    stats = {
        "ad": 40,
        "health": 450,
        "bonus_attack_speed": 0.25
        # active damage = 0.4*ad
    })

infinity_edge = shop_item(
    name="Infinity Edge",
    price = 3450,
    stats = {
        "ad": 65,
        "crit_chance": 0.25
        # +40% crit damage
    })
